In [1]:
%run 전처리.ipynb

## module

In [2]:
import numpy as np
import pandas as pd
import requests; from urllib.parse import urlparse
import folium

## 파일 불러오기

In [3]:
# 서울시 자치구별 인구 데이터
population = pd.read_csv('참고자료/인구.csv', encoding = 'cp949' )
pd.options.display.float_format = '{:.2f}'.format # float display type change
# 인구 데이터 정리 (Year:2020)
population = population.query('기준일ID=="20200530"')
population = population[1:]

In [16]:
# 한강 데이터 불러오기
river = pd.read_csv('참고자료/한강.csv',encoding='cp949')
# 대학교 데이터 불러오기
univ = pd.read_csv('참고자료/대학교.csv', encoding = 'cp949')

In [18]:
 # 스타벅스 자료
cafe = pd.read_csv('참고자료/스타벅스.csv', encoding = 'cp949') 
 # 구별 스타벅스 카운트
cafe1 = pd.read_csv('참고자료/starbucks_cnt.csv', encoding = 'cp949') 

# 전처리

In [78]:
'''data = real_estate
idx = data.index
for i in range(len(data)):
    data.at[idx[i], '거래금액(만원)'] = data['거래금액(만원)'][i].replace(',','')
data['거래금액(만원)'] = data['거래금액(만원)'].astype(int)'''

In [79]:
data

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3
...,...,...,...,...,...,...,...,...,...,...,...,...
1082712,서울특별시 중랑구 중화동,274-75,274,75,한영(102),58.87,202003,17,28800,6,2003,동일로144길 74
1082713,서울특별시 중랑구 중화동,274-77,274,77,한영(104),70.96,202001,28,31200,4,2003,동일로144길 74
1082714,서울특별시 중랑구 중화동,274-77,274,77,한영(104),66.24,202002,3,28500,5,2003,동일로144길 74
1082715,서울특별시 중랑구 중화동,274-77,274,77,한영(104),106.21,202003,11,45400,6,2003,동일로144길 74


## 시군구 분리하기

In [81]:
real_estate['시'],real_estate['구'],real_estate['동'] = real_estate['시군구'].str.split(' ',2).str
real_estate

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,시,구,동
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103,서울특별시,강남구,개포동
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103,서울특별시,강남구,개포동
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,200610,30,"73,500",8,1988,언주로 103,서울특별시,강남구,개포동
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,200612,9,"76,500",5,1988,언주로 103,서울특별시,강남구,개포동
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,79.97,200605,9,"88,000",3,1987,언주로 3,서울특별시,강남구,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082712,서울특별시 중랑구 중화동,274-75,274,75,한영(102),58.87,202003,17,"28,800",6,2003,동일로144길 74,서울특별시,중랑구,중화동
1082713,서울특별시 중랑구 중화동,274-77,274,77,한영(104),70.96,202001,28,"31,200",4,2003,동일로144길 74,서울특별시,중랑구,중화동
1082714,서울특별시 중랑구 중화동,274-77,274,77,한영(104),66.24,202002,3,"28,500",5,2003,동일로144길 74,서울특별시,중랑구,중화동
1082715,서울특별시 중랑구 중화동,274-77,274,77,한영(104),106.21,202003,11,"45,400",6,2003,동일로144길 74,서울특별시,중랑구,중화동


## 제거 [번지Column/도로명' ']

In [82]:
col = ['번지']
real_estate = real_estate.drop(col,axis=1)
# 도로명 빈주소 제거 [ 재건축으로 인한 미존재 아파트]
real_estate

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,시,구,동
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103,서울특별시,강남구,개포동
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103,서울특별시,강남구,개포동
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,"73,500",8,1988,언주로 103,서울특별시,강남구,개포동
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,"76,500",5,1988,언주로 103,서울특별시,강남구,개포동
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,"88,000",3,1987,언주로 3,서울특별시,강남구,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082712,서울특별시 중랑구 중화동,274,75,한영(102),58.87,202003,17,"28,800",6,2003,동일로144길 74,서울특별시,중랑구,중화동
1082713,서울특별시 중랑구 중화동,274,77,한영(104),70.96,202001,28,"31,200",4,2003,동일로144길 74,서울특별시,중랑구,중화동
1082714,서울특별시 중랑구 중화동,274,77,한영(104),66.24,202002,3,"28,500",5,2003,동일로144길 74,서울특별시,중랑구,중화동
1082715,서울특별시 중랑구 중화동,274,77,한영(104),106.21,202003,11,"45,400",6,2003,동일로144길 74,서울특별시,중랑구,중화동


In [83]:
real_estate = real_estate.query('도로명 != " "')
real_estate

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,시,구,동
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103,서울특별시,강남구,개포동
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103,서울특별시,강남구,개포동
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,"73,500",8,1988,언주로 103,서울특별시,강남구,개포동
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,"76,500",5,1988,언주로 103,서울특별시,강남구,개포동
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,"88,000",3,1987,언주로 3,서울특별시,강남구,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082712,서울특별시 중랑구 중화동,274,75,한영(102),58.87,202003,17,"28,800",6,2003,동일로144길 74,서울특별시,중랑구,중화동
1082713,서울특별시 중랑구 중화동,274,77,한영(104),70.96,202001,28,"31,200",4,2003,동일로144길 74,서울특별시,중랑구,중화동
1082714,서울특별시 중랑구 중화동,274,77,한영(104),66.24,202002,3,"28,500",5,2003,동일로144길 74,서울특별시,중랑구,중화동
1082715,서울특별시 중랑구 중화동,274,77,한영(104),106.21,202003,11,"45,400",6,2003,동일로144길 74,서울특별시,중랑구,중화동


## 한강 데이터 전처리

In [139]:
한강 = river.파일경로.str.lower().str.contains('한강') # 한강 문자 포함 행 search
river1 = river[한강]

# 데이터 정교화
list1 = []
list2 = [255, 259, 253, 258, 257, 254, 256, 298, 248, 47, 247, 329, 50, 46, 49, 377,  374, 168, 171, 
53, 62, 52, 60, 51, 61, 59, 330, 81, 82, 336, 90, 173, 92, 48, 180, 310, 328, 23]
# 인덱스에 해당하는 교유번호 추출
list1 = []
for i in list2:
    river_a = river1.at[i, '교유번호']
    list1.append(river_a)
river1 = river1.query('교유번호 not in [181,185,179,184,183,180,182,224,174,14,173,255,17,13,16,304,301,307,309,20,29,19,27,18,28,26,256,48,49,262,57,311,59,15,318,236,254,158]')

# 시각화

In [22]:
# river1
# marker 최근접 한강 포인트
# 인구수별
map = folium.Map(location =[37.5502,126.982], zoom_start =11)

geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_geo_simple.json'

for i in river.index:
    popup_name = i
    folium.Marker([river['위도'][i],river['경도'][i]],
                 popup = popup_name
                 ).add_to(map)

folium.Choropleth(
    geo_data = geo_json,
    name = 'choropleth',
    data = population,
    columns = ['시군구명', '총생활인구수'],
    key_on = 'feature.properties.SIG_KOR_NM',
    fill_color = 'YlOrRd',
    # ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’. [컬러코드]
    fill_opacity = 0.7,
    line_opacity = 1
).add_to(map)    
map

In [20]:
# STARBUCKS 시각화
# marker 스타벅스 위치
# 구별 스타벅스 
map = folium.Map(location =[37.5502,126.982], zoom_start =11) # 서울 중심 위도 경도

geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_geo_simple.json'

for i in cafe.index:
    popup_name = i
    folium.Marker([cafe['위도'][i],cafe['경도'][i]],
                 popup = popup_name
                 ).add_to(map)

folium.Choropleth(
    geo_data = geo_json,
    name = 'choropleth',
    data = cafe1,
    columns = ['gu', 'starbucks_cnt'],
    key_on = 'feature.properties.SIG_KOR_NM',
    fill_color = 'YlOrRd',
    # ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’.
    fill_opacity = 0.7,
    line_opacity = 1
).add_to(map)    
map

In [4]:
population

,기준일ID,시군구코드,시군구명,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,일최대인구수,일최소인구수,주간인구수(09~18),야간인구수(19~08),일최대이동인구수,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수
1,20200530,11110,종로구,268649.60,250902.87,15975.18,1771.54,328024.16,226526.12,306858.29,241357.68,194253.41,48312.16,33503.51,112437.73
2,20200530,11140,중구,226476.70,204347.93,18950.55,3178.21,286393.58,176895.91,264293.10,199464.99,177680.93,42604.39,30475.89,104600.64
3,20200530,11170,용산구,294396.12,269236.45,22365.75,2793.92,307986.98,280171.46,303586.29,287831.72,150257.47,36249.86,49287.60,64720.02
4,20200530,11200,성동구,348750.58,329272.62,16714.57,2763.40,354941.18,342307.14,349868.97,347951.73,164875.08,30242.75,61207.31,73425.02
5,20200530,11215,광진구,404500.63,378029.35,23532.17,2939.11,416267.20,391922.35,396285.63,410368.48,175102.09,36148.67,79663.36,59290.06
6,20200530,11230,동대문구,381345.36,354229.89,24610.57,2504.90,392109.14,367084.65,375373.99,385610.61,166345.09,28074.46,73325.92,64944.71
7,20200530,11260,중랑구,369817.35,362827.81,6330.74,658.79,393851.49,342786.05,350585.62,383554.30,152828.18,25057.36,91560.22,36210.60
8,20200530,11290,성북구,452843.67,433021.08,15808.58,4014.02,474726.67,427000.00,436360.60,464617.30,174322.21,24218.67,86213.93,63889.61
9,20200530,11305,강북구,314385.57,308438.27,5370.84,576.46,332769.28,293213.47,299829.60,324782.70,176629.64,14357.39,118451.22,43821.03
10,20200530,11320,도봉구,289850.07,286465.99,3077.16,306.92,301667.14,275192.97,280520.26,296514.22,135351.11,20170.87,72201.50,42978.74
